In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def subtract_df(df1, df2, on):
    """
    Subtracts the rows of df2 from df1 based on a common column.

    Parameters:
    df1 (pandas.DataFrame): The DataFrame to subtract from.
    df2 (pandas.DataFrame): The DataFrame to subtract.
    on (str or list of str, optional): The column(s) to use as the join key(s). If not specified, the function will use all common columns.

    Returns:
    pandas.DataFrame: The resulting DataFrame after subtracting df2 from df1.
    """
    # merge the DataFrames
    result = df1.merge(df2, on=on, how='left',suffixes=('','_y'), indicator=True)

    # select only the rows in df1
    result = result[result['_merge'] == 'left_only']

    # drop the indicator column
    result = result.drop(['_merge'] + [col for col in result.columns if '_y' in col], axis=1)
                                                                        
    return result


In [4]:
cons_information = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv')
cons_email = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv')
cons_subscription = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv')

In [5]:
# cons_information = pd.read_csv(os.getcwd()+'/cons.csv')
# cons_email = pd.read_csv(os.getcwd()+'/cons_email.csv')
# cons_subscription = pd.read_csv(os.getcwd()+'/cons_email_chapter_subscription.csv')

In [8]:
cons_information.dtypes

cons_id                         int64
prefix                         object
firstname                      object
middlename                     object
lastname                       object
suffix                         object
salutation                     object
gender                         object
birth_dt                       object
title                          object
employer                       object
occupation                     object
income                        float64
source                         object
subsource                      object
userid                          int64
password                       object
is_validated                    int64
is_banned                       int64
change_password_next_login      int64
consent_type_id                 int64
create_dt                      object
create_app                      int64
create_user                     int64
modified_dt                    object
modified_app                    int64
modified_use

In [9]:
cons_email.dtypes

cons_email_id            int64
cons_id                  int64
cons_email_type_id       int64
is_primary               int64
email                   object
canonical_local_part    object
domain                  object
double_validation       object
create_dt               object
create_app               int64
create_user              int64
modified_dt             object
modified_app             int64
modified_user            int64
status                   int64
note                    object
dtype: object

In [10]:
cons_subscription.dtypes

cons_email_chapter_subscription_id     int64
cons_email_id                          int64
chapter_id                             int64
isunsub                                int64
unsub_dt                              object
modified_dt                           object
dtype: object

### Question 1


In [5]:
cons_email_subset = cons_email[['cons_email_id','cons_id','is_primary','email']]

In [6]:
cons_subscription_subset = cons_subscription[['cons_email_id','chapter_id','isunsub']]

In [7]:
cons_email_subset['is_primary'].value_counts()

0    794361
1    605639
Name: is_primary, dtype: int64

In [8]:
# Test
# All emails from subscription is present in email data
subtract_df(cons_subscription_subset,cons_email_subset,on='cons_email_id').shape[0]

0

In [9]:
cons_email_primary = cons_email_subset.loc[cons_email_subset['is_primary']==1].reset_index().drop(['index'],axis=1)

In [10]:
cons_subscription_chap1 = cons_subscription_subset.loc[cons_subscription_subset['chapter_id']==1].reset_index()

In [11]:
# Test
# Single email has subscribed or unsubscribed to chapter 1 only once
sum(cons_subscription_chap1['cons_email_id'].value_counts()>1)

0

In [12]:
cons_subscription_chap1_sub = cons_subscription_subset.loc[(cons_subscription_subset['chapter_id']==1) & (cons_subscription_subset['isunsub']==0)].reset_index()

In [13]:
cons_subscription_chap1_notsub = cons_subscription_subset.loc[(cons_subscription_subset['chapter_id']==1) & (cons_subscription_subset['isunsub']==1)].reset_index()

In [14]:
# Test
# No email that has once subscribed to chapter one and then unsubscribed to chapter 1
cons_subscription_chap1_sub.merge(cons_subscription_chap1_notsub,on='cons_email_id').shape[0]

0

In [15]:
cons_subscription_notchap1 = cons_subscription_subset.loc[cons_subscription_subset['chapter_id']!=1].reset_index()

In [16]:
# Email that are subscribed to chapter 1 as well as other chapters
cons_subscription_commonchap = cons_subscription_notchap1.merge(cons_subscription_chap1_sub, on='cons_email_id',suffixes=('','_y'))
cons_subscription_commonchap = cons_subscription_commonchap.drop([col for col in cons_subscription_commonchap.columns if '_y' in col], axis=1)

In [17]:
cons_subscription_notchap1[cons_subscription_notchap1['cons_email_id']==448407]

,index,cons_email_id,chapter_id,isunsub
2,1772,448407,2,1
1855,52732,448407,4,1
48349,277208,448407,2,1


In [18]:
cons_subscription_chap1[cons_subscription_chap1['cons_email_id']==448407]

,index,cons_email_id,chapter_id,isunsub
946,946,448407,1,0


In [19]:
# Emails that are only subscribed/unsub to chapters other than chapter id one
cons_subscription_notchap1.shape[0]-cons_subscription_commonchap.shape[0]

67115

In [20]:
cons_subscription_notsubchap1 = subtract_df(cons_subscription_notchap1,cons_subscription_commonchap,on='cons_email_id').reset_index().drop(['level_0'],axis=1)

In [21]:
emails_not_sub_chap1_final = pd.concat([cons_subscription_notsubchap1,cons_subscription_chap1_notsub],axis=0)
emails_not_sub_chap1_final = emails_not_sub_chap1_final.drop_duplicates(subset=['cons_email_id'])

In [22]:
# Removing people from email df who present in sub. These people are subscribed to chap 1
email_in_primary_sub_chap1 = subtract_df(cons_email_primary,cons_subscription_subset,on='cons_email_id').reset_index().drop(['is_primary','email','cons_id'], axis=1)
email_in_primary_sub_chap1['chapter_id']=1
email_in_primary_sub_chap1['isunsub']=0

In [23]:
emails_sub_chap1_final = pd.concat([email_in_primary_sub_chap1,cons_subscription_chap1_sub],axis=0).reset_index().drop(['level_0'], axis=1)

In [24]:
peoples_final = pd.concat([emails_sub_chap1_final,emails_not_sub_chap1_final],axis=0).reset_index().drop(['level_0'], axis=1)

In [25]:
people_final_primary_email = cons_email_primary.merge(peoples_final,on='cons_email_id').drop(['index','is_primary','chapter_id'],axis=1)

In [26]:
cons_information_subset = cons_information[['cons_id','source','create_dt','modified_dt']]

In [27]:
final_df =  people_final_primary_email.merge(cons_information_subset,on='cons_id').drop(['cons_email_id','cons_id'],axis=1)

In [28]:
final_df = final_df.rename(columns={'create_dt':'created_dt','modified_dt':'updated_dt','source':'code','isunsub':'is_unsub'})[['email','code','is_unsub','created_dt','updated_dt']]

In [30]:
final_df.to_csv('people.csv',header=True,index=False)

### Question 2

In [32]:
final_df['acquisition_date'] = pd.to_datetime(final_df['created_dt']).dt.date

In [33]:
acquisition_facts = final_df.groupby(final_df['acquisition_date']).agg({'acquisition_date':'count'}).rename(columns={'acquisition_date': 'count'}).reset_index()

In [34]:
acquisition_facts.to_csv('acquisition_facts.csv',header=True,index=False)

In [35]:
acquisition_facts

,acquisition_date,count
0,1970-01-01,28
1,1970-01-02,34
2,1970-01-03,27
3,1970-01-04,36
4,1970-01-05,39
...,...,...
18440,2020-06-27,36
18441,2020-06-28,33
18442,2020-06-29,24
18443,2020-06-30,40
